In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install torchtext==0.4.0

import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True,
                  pad_first=True)
LABEL = data.LabelField(dtype = torch.float)

from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

TEXT.__dict__

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.1 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:17<00:00, 4.88MB/s]


In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc


def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        text, text_lengths = batch.text

        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:
            text, text_lengths = batch.text

            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# LSTM

In [55]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers=1, dropout=0.5):

        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text, text_lengths):
        #text = [sent len, batch size]

        embedded = self.embedding(text)
        #embedded = [sent len, batch size, emb dim]

        output, (hidden, cell) = self.lstm(embedded)
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]

        last_hidden_state = hidden[-1, :, :]

        #assert torch.equal(output[-1,:,:], hidden.squeeze(0))

        return self.fc(last_hidden_state)

In [56]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 500
OUTPUT_DIM = 1

lstm_model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [57]:
print(f'The model lstm_model has {count_parameters(lstm_model):,} trainable parameters')

import torch.optim as optim

optimizer = optim.Adam(lstm_model.parameters(), lr=1e-3)

criterion = nn.BCEWithLogitsLoss()

lstm_model = lstm_model.to(device)
criterion = criterion.to(device)


The model lstm_model has 3,704,701 trainable parameters


In [58]:
N_EPOCHS=50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(lstm_model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(lstm_model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(lstm_model.state_dict(), 'lstm-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

ValueError: Target size (torch.Size([64])) must be the same as input size (torch.Size([132]))

In [ ]:
lstm_model.load_state_dict(torch.load('lstm-model.pt'))

test_loss, test_acc = evaluate(lstm_model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

# CNN

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class CNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, output_dim):

        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=1)

        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=1, kernel_size=3, padding=1)

        self.conv2 = nn.Conv1d(in_channels=1, out_channels=2, kernel_size=3, padding=1)

        self.conv3 = nn.Conv1d(in_channels=2, out_channels=3, kernel_size=3, padding=1)

        # self.convs = nn.ModuleList([self.conv1, self.conv2, self.conv3])

        # self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        # self.fc1 = nn.Linear(32 * 8 * 8, 128) 
        # self.fc2 = nn.Linear(128, output_dim)
        self.fc = nn.Linear(3, output_dim)
        

    def forward(self, text, text_lengths):
        #text = [sent len, batch size]

        embedded = self.embedding(text)
        embedded = embedded.permute(0, 2, 1)
        # [batch, emb_dim, seq_len, 1]
        #embedded = [sent len, batch size, emb dim]

        output1 = F.relu(self.conv1(embedded))
        output2= F.relu(self.conv2(output1))
        output3= F.relu(self.conv3(output2))

        # Create a mask based on text_lengths
        batch_size = text.size(0)

        mask = torch.arange(text. size(1), device=text.device).unsqueeze(0) < text_lengths.unsqueeze(1)

        output3 = (output3 * mask).unsqueeze(1).float()
        
        # Global average pooling - average over actual sequence length (not padding)
        # Sum over time dimension
        pooled = output3.sum(dim=2)  # [batch_size, num_filters]

        # Divide by actual lengths to get mean
        pooled = pooled / text_lengths.unsqueeze(1).float()

        pooled = self.dropout(pooled)

        # pooled = F.adaptive_avg_pool1d(output3, 1)

        # pooled = pooled.view(pooled.size(0), -1)

        # pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        # cat = self.dropout(torch.cat(pooled, dim=1))

        # x = torch.flatten(output3, 1)
        # x = F.relu(self.fc1(x))

        #assert torch.equal(output[-1,:,:], hidden.squeeze(0))

        return self.fc(pooled)

In [27]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1

cnn = CNN(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM)

In [28]:
print(f'The model has {count_parameters(cnn):,} trainable parameters')

import torch.optim as optim

optimizer = optim.Adam(cnn.parameters(), lr=1e-3)

criterion = nn.BCEWithLogitsLoss()

cnn = cnn.to(device)
criterion = criterion.to(device)


The model has 2,500,534 trainable parameters


In [29]:
N_EPOCHS=50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(cnn, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(cnn, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(cnn.state_dict(), 'cnn-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

RuntimeError: The size of tensor a (3) must match the size of tensor b (64) at non-singleton dimension 1

In [ ]:
cnn.load_state_dict(torch.load('cnn-model.pt'))

test_loss, test_acc = evaluate(cnn, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')